# Ejemplo STDP: Aprendizaje Biológico

**Descripción:** Tutorial interactivo sobre el mecanismo de aprendizaje biológico STDP (Plasticidad Dependiente del Tiempo de Disparo) utilizado en redes neuronales neuromórficas. Demuestra cómo las neuronas aprenden correlaciones temporales automáticamente.

**Autor:** Mauro Risonho de Paula Assumpção.
**Fecha de creación:** 5 de diciembre de 2025.
**Licencia:** MIT License.
**Desarrollo:** Desarrollo asistido Humano + IA (Claude Sonnet 4.5, Gemini 3 Pro Preview).

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
# Instalar la librería Brian2 si aún no está instalada
try:
    import brian2
except ImportError:
    !pip install brian2
    import brian2

# Importaciones específicas de brian2 en lugar de comodines
from brian2 import (
    ms, mV, Hz, second,
    NeuronGroup, Synapses, SpikeMonitor, StateMonitor,
    SpikeGeneratorGroup, Network,
    defaultclock, run, device, start_scope,
    clip, prefs
)

# Configurar para usar numpy (evita errores de compilación C++ si faltan cabeceras)
prefs.codegen.target = "numpy"

plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

print("✓ ¡Importaciones completadas!")

# STDP: Plasticidad Dependiente del Tiempo de Disparo

**Descripción:** Tutorial interactivo sobre el mecanismo de aprendizaje biológico STDP (Plasticidad Dependiente del Tiempo de Disparo) utilizado en redes neuronales neuromórficas. Demuestra cómo las neuronas aprenden correlaciones temporales automáticamente.

**Autor:** Mauro Risonho de Paula Assumpção
**Fecha de creación:** 5 de diciembre de 2025
**Licencia:** MIT License
**Desarrollo:** Desarrollo asistido Humano + IA (Claude Sonnet 4.5, Gemini 3 Pro Preview).

---

Este cuaderno explora el mecanismo de aprendizaje biológico **STDP** utilizado en redes neuronales neuromórficas.

## ¿Qué es STDP?

STDP (Plasticidad Dependiente del Tiempo de Disparo) es una regla de aprendizaje **no supervisado** inspirada en neuronas biológicas:

- **si la neurona pre-sináptica dispara ANTES que la post-sináptica** → **Potenciación** (peso ↑)
- **si la neurona pre-sináptica dispara DESPUÉS que la post-sináptica** → **Depresión** (peso ↓)

Esto permite que la red aprenda **relaciones causales temporales** sin etiquetas explícitas.

## 1. Configuración e Importaciones

## 1. Curva STDP clásica

Visualiza cómo el cambio en el peso depende de la diferencia temporal entre los disparos.

In [ ]:
# Parámetros de STDP
tau_pre = 20.0 # ms - constante de tiempo pre-sináptica
tau_post = 20.0 # ms - constante de tiempo post-sináptica
A_pre = 0.01 # Amplitud de potenciación
A_post = -0.012 # Amplitud de depresión

# Delta t (diferencia de tiempo)
dt_range = np.linspace(-100, 100, 500) # ms

# Calcular cambio de peso
def stdp_weight_change(dt, tau_pre, tau_post, A_pre, A_post):
    """
    Calcula el cambio de peso vía STDP.
    dt = t_post - t_pre
    """
    if dt > 0: # Post después de Pre → Potenciación
        return A_pre * np.exp(-dt / tau_pre)
    else: # Post antes de Pre → Depresión
        return A_post * np.exp(dt / tau_post)

weight_changes = np.array([stdp_weight_change(dt, tau_pre, tau_post, A_pre, A_post)
                           for dt in dt_range])

# Graficar la curva STDP
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(dt_range, weight_changes, linewidth=3, color='purple')
ax.axhline(0, color='black', linestyle='--', linewidth=1, alpha=0.5)
ax.axvline(0, color='black', linestyle='--', linewidth=1, alpha=0.5)

# Anotar regiones
ax.fill_between(dt_range[dt_range > 0], 0, weight_changes[dt_range > 0],
                alpha=0.2, color='green', label='Potenciación (LTP)')
ax.fill_between(dt_range[dt_range < 0], 0, weight_changes[dt_range < 0],
                alpha=0.2, color='red', label='Depresión (LTD)')

ax.set_xlabel('Δt = t_post - t_pre (ms)', fontsize=12)
ax.set_ylabel('Cambio de peso (Δw)', fontsize=12)
ax.set_title('Curva STDP: Plasticidad Dependiente del Tiempo de Disparo', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# Añadir anotaciones
ax.annotate('Pre → Post
(Causal)', xy=(20, 0.008), fontsize=10,
            ha='center', color='green', fontweight='bold')
ax.annotate('Post → Pre
(Anti-causal)', xy=(-20, -0.009), fontsize=10,
            ha='center', color='red', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n✓ Interpretación:")
print("  - Δt > 0: la neurona pre-sináptica dispara ANTES → Potenciación (fortalece la conexión)")
print("  - Δt < 0: la neurona pre-sináptica dispara DESPUÉS → Depresión (debilita la conexión)")
print("  - El efecto decae exponencialmente con |Δt|")

## 2. Simulación STDP con Brian2

Simula dos neuronas conectadas con STDP y observa la evolución del peso.

In [ ]:
start_scope()

# Parámetros de simulación
duration = 100*ms  # type: ignore[operator]
defaultclock.dt = 0.1*ms  # type: ignore[operator]

print("⚙️ Configurando la simulación STDP...")
print(f"Duración: {duration}")
print(f"Paso de tiempo: {defaultclock.dt}\n")

# Neuronas LIF
tau_m = 10*ms  # type: ignore[operator]
tau_syn = 5*ms  # type: ignore[operator] # Constante de tiempo sináptica
v_rest = -70*mV
v_thresh = -50*mV
v_reset = -70*mV

# Añadir decaimiento sináptico (dI_syn/dt)
eqs_post = '''
dv/dt = (v_rest - v + I_syn) / tau_m : volt
dI_syn/dt = -I_syn / tau_syn : volt
'''

# Crear neuronas
neuron_pre = SpikeGeneratorGroup(1, [0], [10]*ms)  # type: ignore[operator]
neuron_post = NeuronGroup(1, eqs_post, threshold='v > v_thresh',
                          reset='v = v_reset', method='euler')
neuron_post.v = v_rest
neuron_post.I_syn = 0*mV

# Parámetros STDP
tau_pre_stdp = 20*ms  # type: ignore[operator]
tau_post_stdp = 20*ms  # type: ignore[operator]
A_pre_stdp = 0.01
A_post_stdp = -0.012
w_max = 1.0
w_min = 0.0

synapse_model = '''
w : 1
dApre/dt = -Apre / tau_pre_stdp : 1 (event-driven)
dApost/dt = -Apost / tau_post_stdp : 1 (event-driven)
'''

# Aumentar ganancia sináptica para asegurar disparo (w * 60*mV)
on_pre_stdp = '''
I_syn_post += w * 60 * mV
Apre += A_pre_stdp
w = clip(w + Apost, w_min, w_max)
'''

on_post_stdp = '''
Apost += A_post_stdp
w = clip(w + Apre, w_min, w_max)
'''

synapse = Synapses(neuron_pre, neuron_post,
                   model=synapse_model,
                   on_pre=on_pre_stdp,
                   on_post=on_post_stdp,
                   method='euler')
synapse.connect(i=0, j=0)
synapse.w = 0.5 # Peso inicial

# Monitores
mon_pre = SpikeMonitor(neuron_pre)
mon_post = SpikeMonitor(neuron_post)
mon_weight = StateMonitor(synapse, 'w', record=True)
mon_voltage = StateMonitor(neuron_post, 'v', record=True)

# Ejecutar simulación
print("⏳ Ejecutando simulación Brian2...")
import time
start_time = time.time()

net = Network(neuron_pre, neuron_post, synapse, mon_pre, mon_post, mon_weight, mon_voltage)
net.run(duration)

sim_time = time.time() - start_time

print(f"✅ ¡Simulación completada en {sim_time:.3f}s!")
print(f"\n Resultados:")
print(f" Disparos pre-sinápticos: {len(mon_pre.t)}")
print(f" Disparos post-sinápticos: {len(mon_post.t)}")
print(f" Peso inicial: {0.5:.3f}")
print(f" Peso final: {mon_weight.w[0][-1]:.3f}")
print(f" Cambio: {(mon_weight.w[0][-1] - 0.5):.3f} ({(mon_weight.w[0][-1] - 0.5)/0.5*100:+.1f}%)")

In [ ]:
# Visualizar resultados
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# Gráfico 1: Disparos
if len(mon_pre.t) > 0:
    axes[0].eventplot([mon_pre.t/ms], lineoffsets=1, linelengths=0.8,
                      linewidths=2, colors='blue', label='pre-sináptico')
if len(mon_post.t) > 0:
    axes[0].eventplot([mon_post.t/ms], lineoffsets=0, linelengths=0.8,
                      linewidths=2, colors='red', label='post-sináptico')

axes[0].set_ylabel('neurona')
axes[0].set_yticks([0, 1])
axes[0].set_yticklabels(['Post', 'Pre'])
axes[0].set_title('Gráfico Raster: Disparos Pre y Post-Sinápticos', fontsize=12, fontweight='bold')
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)

# Gráfico 2: Evolución del peso sináptico
axes[1].plot(mon_weight.t/ms, mon_weight.w[0], linewidth=2.5, color='purple')
axes[1].axhline(0.5, color='gray', linestyle='--', alpha=0.5, label='Peso inicial')
axes[1].set_ylabel('Peso sináptico (w)')
axes[1].set_title('Evolución del peso sináptico con STDP', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Gráfico 3: Voltaje de la neurona post-sináptica
axes[2].plot(mon_voltage.t/ms, mon_voltage.v[0]/mV, linewidth=1.5, color='green')
axes[2].axhline(-50, color='red', linestyle='--', alpha=0.7, label='Umbral')
axes[2].axhline(-70, color='gray', linestyle='--', alpha=0.5, label='Reposo')
axes[2].set_xlabel('tiempo (ms)')
axes[2].set_ylabel('Voltaje (mV)')
axes[2].set_title('Potencial de membrana post-sináptico', fontsize=12, fontweight='bold')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. STDP con Patrones de Entrada

Demuestra cómo STDP aprende correlaciones temporales en patrones repetidos.

In [ ]:
start_scope()

# Simular múltiples neuronas pre-sinápticas
n_pre = 5
n_post = 1
duration = 500*ms  # type: ignore[operator]
defaultclock.dt = 0.1*ms  # type: ignore[operator]

print("⚙️ Configurando simulación con múltiples neuronas...")
print(f"Neuronas pre-sinápticas: {n_pre}")
print(f"Duración: {duration}\n")

# Generar patrón temporal (algunas neuronas disparan en secuencia)
spike_pattern = [
    [10, 110, 210, 310, 410], # neurona 0: disparos regulares
    [15, 115, 215, 315, 415], # neurona 1: ligeramente retrasada
    [20, 120, 220, 320, 420], # neurona 2: más retrasada
    [100, 200, 300, 400], # neurona 3: disparos dispersos
    [50, 150, 250, 350, 450] # neurona 4: fase diferente
]

indices = []
times = []
print("📊 Patrones de disparo:")
for neuron_idx, spike_times in enumerate(spike_pattern):
    print(f"  Neurona {neuron_idx}: {len(spike_times)} disparos")
    for t in spike_times:
        indices.append(neuron_idx)
        times.append(t)

neuron_pre = SpikeGeneratorGroup(n_pre, indices, times*ms)  # type: ignore[operator]

# Neurona post-sináptica
neuron_post = NeuronGroup(n_post, eqs_post, threshold='v > v_thresh',
                          reset='v = v_reset', method='euler')
neuron_post.v = v_rest

# Sinapsis con STDP
synapse = Synapses(neuron_pre, neuron_post,
                   model=synapse_model,
                   on_pre=on_pre_stdp,
                   on_post=on_post_stdp,
                   method='euler')
synapse.connect() # Conectar todas
synapse.w = 'rand() * 0.3 + 0.2' # Pesos iniciales aleatorios [0.2, 0.5]

# Monitores
mon_pre = SpikeMonitor(neuron_pre)
mon_post = SpikeMonitor(neuron_post)
mon_weight = StateMonitor(synapse, 'w', record=True)

# Guardar pesos iniciales
initial_weights = np.array(synapse.w).copy()

# Ejecutar
print("\n⏳ Ejecutando simulación de patrones temporales...")
start_time = time.time()

net = Network(neuron_pre, neuron_post, synapse, mon_pre, mon_post, mon_weight)
net.run(duration)

sim_time = time.time() - start_time
final_weights = np.array(synapse.w).copy()

print(f"✅ ¡Simulación completada en {sim_time:.3f}s!")
print(f"\n📊 Análisis de pesos sinápticos:")
for i in range(n_pre):
    delta = final_weights[i] - initial_weights[i]
    percentage = (delta / initial_weights[i]) * 100 if initial_weights[i] > 0 else 0
    print(f"  Neurona {i}: {initial_weights[i]:.3f} → {final_weights[i]:.3f} (Δ = {delta:+.3f}, {percentage:+.1f}%)")

In [ ]:
# Visualizar la evolución de los pesos
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Gráfico 1: Evolución temporal de los pesos
for i in range(n_pre):
 axes[0].plot(mon_weight.t/ms, mon_weight.w[i], label=f'Sinapsis {i}', linewidth=2)

axes[0].set_xlabel('Tiempo (ms)')
axes[0].set_ylabel('Peso sináptico')
axes[0].set_title('Evolución temporal de los pesos sinápticos con STDP', fontsize=12, fontweight='bold')
axes[0].legend(loc='best')
axes[0].grid(True, alpha=0.3)

# Gráfico 2: Comparación antes/después
x_pos = np.arange(n_pre)
width = 0.35

axes[1].bar(x_pos - width/2, initial_weights, width, label='Inicial', alpha=0.7, color='lightblue')
axes[1].bar(x_pos + width/2, final_weights, width, label='Final', alpha=0.7, color='darkblue')

axes[1].set_xlabel('Neurona pre-sináptica')
axes[1].set_ylabel('Peso sináptico')
axes[1].set_title('Comparación: Pesos iniciales vs finales', fontsize=12, fontweight='bold')
axes[1].set_xticks(x_pos)
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n💡 Interpretación:")
print("  - Las neuronas que disparan consistentemente ANTES que la post-sináptica se refuerzan")
print("  - Las neuronas con tiempos inconsistentes reducen sus pesos")
print("  - ¡La red aprende correlaciones temporales automáticamente!")

## 4. Aplicación a la Detección de Fraude

¿Cómo ayuda STDP en la detección de fraude?

### Escenario 1: Secuencia Temporal Normal

**Transacción legítima:**
1. Inicio de sesión en la app (t=0ms)
2. Navegación al saldo (t=500ms)
3. Selección de beneficiario conocido (t=2000ms)
4. Confirmación de pago (t=3000ms)

**STDP aprende:**
- Secuencia causal esperada
- Intervalos temporales normales
- Refuerza conexiones que representan comportamiento legítimo

### Escenario 2: Secuencia Anómala (Fraude)

**Transacción fraudulenta:**
1. Inicio de sesión en la app (t=0ms)
2. Transferencia inmediata sin navegación (t=50ms)
3. Alto valor a nuevo beneficiario (t=100ms)
4. Ubicación geográfica inconsistente (t=150ms)

**STDP detecta:**
- Patrón temporal anómalo
- Secuencia no reforzada durante el entrenamiento
- Alta activación de neuronas "de fraude"

### Ventajas de STDP:

1. **Aprendizaje no supervisado**: No requiere etiquetas explícitas inicialmente
2. **Adaptación continua**: Aprende nuevos patrones de fraude automáticamente
3. **Sensibilidad temporal**: Detecta anomalías en secuencias de eventos
4. **Eficiencia**: Actualizaciones de peso locales (sin retropropagación)
5. **Plausible biológicamente**: Inspirado en el cerebro humano

## 5. Conclusiones

### STDP en la Detección de Fraude

**Mecanismo:**
- Aprende correlaciones temporales entre características de la transacción
- Refuerza patrones legítimos frecuentes
- Detecta desviaciones en secuencias temporales

**Aplicaciones prácticas:**
1. **Análisis de comportamiento**: Secuencia de acciones en banca móvil
2. **Detección de velocidad**: Transacciones imposibles (p. ej., compras en ciudades distintas en minutos)
3. **Patrones de uso**: Horarios, frecuencia, valores típicos
4. **Navegación sospechosa**: Secuencias de páginas atípicas

**Comparación con métodos tradicionales:**

| Característica | STDP/SNN | DNN/LSTM |
|----------------|----------|----------|
| Procesamiento temporal | Nativo | Emulado |
| Supervisión | No | Sí |
| Latencia | Ultra baja (~ms) | Alta (~100ms) |
| Consumo de energía | Muy bajo | Alto |
| Adaptación en línea | Sí | Difícil |
| Hardware especializado | Sí (Loihi, TrueNorth) | GPU |

### Direcciones futuras

- Chips neuromórficos dedicados (Intel Loihi 2, IBM NorthPole)
- STDP + Modulación por recompensa (dopamina artificial)
- Aprendizaje federado con STDP
- Explicabilidad: Visualizar pesos aprendidos

---

**Autor:** Mauro Risonho de Paula Assumpção
**Proyecto:** Computación Neuromórfica para la Ciberseguridad Bancaria